In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/CS 577 Project/helper functions
%ls

import cv2 as cv
import os
from skimage import io, transform
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms#, utils
# import torch.optim as optim

import utility as utils
from video_utility import create_video_from_intermediate_results

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import glob

from data_loader import RescaleT
from data_loader import ToTensor
from data_loader import ToTensorLab
from data_loader import SalObjDataset

from models import U2NET # full size version 173.6 MB
from models import U2NETP # small version u2net 4.7 MB


PATH = '/content/drive/My Drive/CS 577 Project'

%cd /content/drive/My Drive/CS 577 Project/
%ls

import U2Net as u2net
import StyleTransfer as style_transfer

/content/drive/My Drive/CS 577 Project/helper functions
data_loader.py  models.py  __pycache__/  utility.py  vgg_nets.py  video_utility.py
/content/drive/My Drive/CS 577 Project
'Code Reference.txt'     images/                  saved_models/         U2Net.ipynb
'CS 577 Project.ipynb'   Image_Similarity.ipynb   StyleTransfer.ipynb   U2Net.py
 Grabcut.ipynb           output_experiment/       StyleTransfer.py
'helper functions'/      __pycache__/             test_data/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def compute_image_mask(model_name, image_dir, prediction_dir, model_dir, figure_name):

    u2net.main(model_name, image_dir, prediction_dir, model_dir, figure_name)


In [ ]:
def transfer_style(config):

    style_transfer.neural_style_transfer(config)


In [ ]:
def main(config):

    compute_image_mask(config['model_name'], config['content_images_dir'], config['output_img_dir'], config['model_dir'], config['content_img_name'])
    dump_path = transfer_style(config)
    print(dump_path)
    combine_final_img(config['content_img_name'])

In [ ]:
def combine_final_img(figure_name):

    name_len = len(figure_name)
    figure_name_new = figure_name[:name_len-4]  # remove the image extension.

    mask_img_path = PATH + '/test_data/images/u2net_results/' + figure_name_new + '.png'
    og_img_path = PATH + '/test_data/images/content/' + figure_name_new + '.jpg'
    style_path = PATH + '/test_data/images/style output/'+'stylized_'+ config['content_img_name']

    #save_out_path = PATH + '/test_data/images/final_output/foreground/' + figure_name_new + '_fg.png'
    #save_bckgr_path = PATH + '/test_data/images/final_output/background/'+ figure_name_new + '_bg.png'
    save_final_image = PATH + '/test_data/images/final output/' + figure_name_new + '_final.png'

    #mask image
    subimage=Image.open(mask_img_path)
    #original image
    original=Image.open(og_img_path)
    #background image
    styleimage=Image.open(style_path)
    #final image
    finalimage=Image.open(og_img_path)

    subimage.convert("RGBA")
    original=original.convert("RGBA")
    styleimage=original.convert("RGBA")
    finalimage=original.convert("RGBA")

    subdata=subimage.getdata()
    ogdata=original.getdata()
    styledata=styleimage.getdata()

    #newdata=[]
    final_data=[]


    for i in range(subdata.size[0]*subdata.size[1]):
      if subdata[i][0]==0 and subdata[i][1]==0 and subdata[i][2]==0:
        #newdata.append((255,255,255,0))
        final_data.append(ogdata[i])
      else:
        #newdata.append(ogdata[i])
        final_data.append(styledata[i])

    #subimage.putdata(newdata)
    #subimage.save(save_out_path,"PNG")


    finalimage.putdata(final_data)
    finalimage.save(save_final_image,"PNG")

    #return subimage, bckgrimage

In [ ]:
# --------- 1. get image path and name ---------

input_image = 'parrot.jpg'
style_image = 'candy.jpg'

model_name='u2net'   #'u2netp'
image_dir=os.path.join(os.getcwd(),'test_data','images','content')
style_dir=os.path.join(os.getcwd(),'test_data','images','style')
prediction_dir=os.path.join(os.getcwd(),'test_data','images',model_name+'_results'+os.sep)
model_dir = os.path.join(os.getcwd(),'saved_models',model_name,model_name+'.pth')
style_output_dir = os.path.join(os.getcwd(), '/content/drive/My Drive/CS 577 Project/test_data/images/style output/')
output_dir = os.path.join(os.getcwd(), '/content/drive/My Drive/CS 577 Project/test_data/images/final output/')



config = {'content_images_dir': image_dir,                                              # where the images are saved. '/content/drive/My Drive/CS 577 Project/test_data/images/content/'
          'content_img_name': input_image,
          'style_images_dir': style_dir,                                                # where the images are saved. '/content/drive/My Drive/CS 577 Project/test_data/images/style/'
          'style_img_name': style_image,
          'output_img_dir': output_dir,                                                 # where to output images.  '/content/drive/My Drive/CS 577 Project/images/output/'
          'u2net_pred_dir': prediction_dir,                                             # where to output the masks from u2net.
          'style_output_dir':style_output_dir,
          'model_dir': model_dir,
          'height': 512,                                                                # (height, width) of the image.
          'init_method': 'content',                                                     # what to use for as content image.
          'model': 'vgg16',                                                             # which NN model to use.
          'optimizer': 'lbfgs',                                                         # which optimizer to use.
          'iter_num': {"lbfgs": 200,"adam": 100},                                        # number of iterations we want to run.
          'content_weight': 1e4,                                                        # weights for the loss function - controls the influence of the content image on the final image.
          'style_weight': 1e4,                                                          # weights for the loss function - controls the influence of the style image on the final image.
          'tv_weight': 5e0,                                                             # total varionation weight for the loss function - controls the smoothness between content and style images.
          'saving_freq': -1,                                                            # how often to save the output of the network.
          'img_format': ('combined', '.jpg'),
          'model_name': 'u2net'}



In [ ]:
main(config)

['/content/drive/MyDrive/CS 577 Project/test_data/images/content/5.jpg', '/content/drive/MyDrive/CS 577 Project/test_data/images/content/figures.jpg', '/content/drive/MyDrive/CS 577 Project/test_data/images/content/parrot.jpg']
/content/drive/MyDrive/CS 577 Project/test_data/images/content/parrot.jpg
...load U2NET---173.6 MB
inferencing: parrot.jpg
saving mask image - NOT WORKING
Using vgg16 in the optimization procedure.
number of iterations:  200
Using LBFGS
L-BFGS | iteration: 000, total loss=856649170944.0000, content_loss=      0.0000, style loss=856604080000.0000, tv loss=45081835.0000
L-BFGS | iteration: 001, total loss=856649105408.0000, content_loss=      0.0000, style loss=856604000000.0000, tv loss=45081835.0000
L-BFGS | iteration: 002, total loss=856649105408.0000, content_loss=      0.0000, style loss=856604000000.0000, tv loss=45081835.0000
L-BFGS | iteration: 003, total loss=856649105408.0000, content_loss=      0.0000, style loss=856604000000.0000, tv loss=45081835.0000